In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)
print(f"redz bef : {utils.stats(redz)}")

# resample to increase the number of binaries
mod_resamp = holo.PM_Resample(resample=5.0)
# modify population (in-place)
pop.modify(mod_resamp)

redz = cosmo.a_to_z(pop.scafa)
print(f"redz aft : {utils.stats(redz)}")


In [ ]:
hards = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(),
]

evo = holo.evolution.Evolution(pop, hards, debug=True)

In [ ]:
log.setLevel(logging.WARNING)

In [ ]:
evo.evolve()

In [ ]:
# utils.stats(evo._dadt_2)
# evo.dadt[0]
# evo._dadt_1[0]

In [ ]:
rads = np.logspace(-5, 4, 100) * PC
data = evo.at('sepa', rads, pars=['dadt', '_dadt_0', '_dadt_1', '_dadt_2'], lin_interp=True)
dadt = data['dadt']

In [ ]:
fig, ax = utils.figax()

vals = np.fabs(dadt)
vals = rads / vals
vals = vals / YR

confs = utils.quantiles(vals, sigmas=[-1, 0, 1], axis=0)

confs = confs.T
med, *confs = confs[[1, 0, 2]]

xx = rads / PC
ax.plot(xx, med)
ax.fill_between(xx, *confs, alpha=0.2)

for ii in range(3):
    vals = np.fabs(data[f"_dadt_{ii}"])
        
    vals = rads / vals
    vals = vals / YR

    confs = utils.quantiles(vals, sigmas=[-1, 1], axis=0).T
    ax.fill_between(xx, *confs, alpha=0.1, label=ii)

frac = utils.frac_str(evo.scafa[:, -1] < 1.0)
print(frac)

ax.set(title=f"coalescing = {frac}", ylim=[1e-2, 1e12])

plt.legend()
plt.show()